In [23]:
import joblib
import time 

import pandas as pd
import numpy as np
import random
import properscoring as ps
from datetime import datetime

import joblib
from scipy.stats import norm
import os
from sklearn.metrics import r2_score

from typing import List, NamedTuple
import sys

sys.path.append("./")
sys.path.append("../")
sys.path.append("../..")
from utils.tabnet_proba import TabNetRegressorProba

import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [7]:
input_cols = [
    "gen_other",
    "gen_solar",
    "gen_wind_on",
    "gen_waste",
    "gen_nuclear",
    "gen_biomass",
    "gen_gas",
    "gen_run_off_hydro",
    "gen_oil",
    "gen_pumped_hydro",
    "gen_other_renew",
    "gen_reservoir_hydro",
    "gen_hard_coal",
    "gen_wind_off",
    "gen_geothermal",
    "gen_lignite",
    "load",
    "gen_coal_gas",
    "total_gen",
    "synchronous_gen",
    "load_ramp",
    "total_gen_ramp",
    "other_ramp",
    "solar_ramp",
    "wind_on_ramp",
    "waste_ramp",
    "nuclear_ramp",
    "biomass_ramp",
    "gas_ramp",
    "run_off_hydro_ramp",
    "oil_ramp",
    "pumped_hydro_ramp",
    "other_renew_ramp",
    "reservoir_hydro_ramp",
    "hard_coal_ramp",
    "wind_off_ramp",
    "geothermal_ramp",
    "lignite_ramp",
    "coal_gas_ramp",
    "forecast_error_wind_on",
    "forecast_error_wind_off",
    "forecast_error_solar",
    "forecast_error_total_gen",
    "forecast_error_load",
    "forecast_error_load_ramp",
    "forecast_error_total_gen_ramp",
    "forecast_error_wind_off_ramp",
    "forecast_error_wind_on_ramp",
    "forecast_error_solar_ramp",
    "solar_day_ahead",
    "wind_on_day_ahead",
    "scheduled_gen_total",
    "prices_day_ahead",
    "load_day_ahead",
    "wind_off_day_ahead",
    "month",
    "weekday",
    "hour",
    "load_ramp_day_ahead",
    "total_gen_ramp_day_ahead",
    "wind_off_ramp_day_ahead",
    "wind_on_ramp_day_ahead",
    "solar_ramp_day_ahead",
    "price_ramp_day_ahead",
    "gen_fossil_peat",
    "fossil_peat_ramp",
    "residual",
]


input_col_names = [
    "Generation other",
    "Solar generation",
    "Onshore wind generation",
    "Waste generation",
    "Nuclear generation",
    "Biomass generation",
    "Gas generation",
    "Run-off-river hydro generation",
    "Oil generation",
    "Pumped hydro generation",
    "Other renewable generation",
    "Reservoir hydro generation",
    "Hard coal generation",
    "Wind offshore generation",
    "Geothermal generation",
    "Lignite generation",
    "Load",
    "Coal gas generation",
    "Total generation",
    "Synchronous generation",
    "Load ramp",
    "Total generation ramp",
    "Other ramp",
    "Solar ramp",
    "Onshore wind ramp",
    "Waste ramp",
    "Nuclear ramp",
    "Biomass ramp",
    "Gas ramp",
    "Run-off-river hydro ramp",
    "Oil ramp",
    "Pumped hydro ramp",
    "Other renewable ramp",
    "Reservoir hydro ramp",
    "Hard coal ramp",
    "Offshore wind ramp",
    "geothermal_ramp",
    "Lignite ramp",
    "Coal gas ramp",
    "Forecast error onshore wind",
    "Forecast error offshore wind",
    "Forecast error solar",
    "Forecast error total generation",
    "Forecast error load",
    "Forecast error load ramp",
    "Forecast error generation ramp",
    "Forecast error offshore wind ramp",
    "Forecast error onshore wind ramp",
    "Forecast error solar ramp",
    "Solar day-ahead",
    "Onshore wind day-ahead",
    "Scheduled generation",
    "Prices day-ahead",
    "Load day-ahead",
    "Offshore wind day-ahead",
    "Month",
    "Weekday",
    "Hour",
    "Load ramp day-ahead",
    "Generation ramp day-ahead",
    "Offshore wind ramp day-ahead",
    "Onshore wind ramp day-ahead",
    "Solar ramp day-ahead",
    "Price ramp day-ahead",
    "Fossil peat generation",
    "Fossil peat ramp",
    "Residual",
]

input_col_names = dict(zip(input_cols, input_col_names))
#input_col_names_units = dict(zip(input_cols, input_col_names_units))
#input_col_names_units_general = dict(zip(input_cols, input_col_names_units_general))

input_rescale_factors = pd.Series(index=input_cols, data=1 / 1000)
input_rescale_factors.loc[
    ["weekday", "hour", "month", "prices_day_ahead", "price_ramp_day_ahead"]
] = 1


In [8]:
class Config(NamedTuple):
    data_version: str = "2024-05-19"
    res_version: str = "2024-06-24"
    model_type: str = "_full"
    model_combination: str = "ngb"
    model_name: str = "NGBoost"
    predictors_prob: List[str] = ["baseline", "predictions"]
    model_names_prob: List[str] =["Baseline", "NGBoost (full model)"],
    model_names_det: List[str] =["Daily profile", "NGBoost (full model)"],    
    predictors_det: List[str] = ["daily_profile", "predictions"]
    scaler_str: str = "yeo_johnson"
    scaled: str = "_scaled"
    explanations: str ="_partition" # for tabnet proba it can be "_partition" because of the two explainers

config_ngb = Config()

config_tabnet_proba = Config(
    res_version="2024-08-27",
    model_combination="tabnet_proba_final",
    model_name = "TabnetProba",
    model_names_det=["Daily profile", "TabNetProba"],
    explanations="_partition"
)

config_xgb = Config(
    res_version = "2024-05-19",
    model_combination= "xgb",
    model_name = "XGBoost",
    predictors_det = ["daily_profile", "predictions"],
    model_names_det = ["Daily profile", "XGBoost"],
    scaler_str = "",
    scaled = "",
    explanations="_partition"
)

config_tabnet = Config(
    res_version = "2024-09-20",
    model_combination= "tabnet_det_rs",
    model_name = "TabNet",
    predictors_det = ["daily_profile", "predictions"],
    model_names_det = ["Daily profile", "TabNet"],
    scaler_str = "yeo_johnson",
    scaled = "_scaled",
    explanations="_partition"
)


In [9]:
data_folder = "../data/2020-2024/{}/version_{}/yeo_johnson/"
explain_folder = "../explanations/{}/version_{}_{}/target_{}/explanations{}/"
fit_folder = "../results/model_fit/{}/version_{}_{}"+ "/{}/" + "target_{}/"

scaled_str = "_scaled"

In [10]:
area_names = ["Continental Europe", "Nordic"]
area_colors = ["C0", "C1"]
model_colors = ['#DF9B1B', '#009682', '#4664AA', '#8CB6C3',   '#A22223', '#20b2aa']
targets = ["f_rocof", "f_ext", "f_msd", "f_integral"]
target_names = {"f_rocof": "RoCoF",
                "f_ext": "Nadir",
                "f_msd": "MSD",
                "f_integral": "Integral"}
target_units = {"f_rocof": "Hz/s",
                "f_ext": "Hz",
                "f_msd": "",
                "f_integral": ""}

# Figure 1 - Frequency Time Series

# Figure 2. Performance Comparison of Deterministc Models

## TODO: Create Function and also use for figure 12

In [ ]:

# Set LaTeX for rendering text in Matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
# Enable LaTeX text rendering and set a larger default font size
plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=16)  # Increase font size
plt.rc('axes', titlesize=20)  # Title font size
plt.rc('axes', labelsize=18)  # X and Y axis label font size
plt.rc('legend', fontsize=14)  # Legend font size
plt.rc('xtick', labelsize=16)  # X-axis tick label size
plt.rc('ytick', labelsize=16)  # Y-axis tick label size

configs = [
    config_xgboost,
    config_tabnet,
    config_ngb,
    #config_tabnet_det,
    
    config_tabnet_proba,
]

# Define paths
version_folder = "../../data/2020-2024/{}/version_{}/" + config_ngb.scaler_str + "/"
version_folder_xgb = "../../data/2020-2024/{}/version_{}/"

fit_folder_template = (
    "../../results/model_fit/{}/version_{}_" + "{}" + "/" + "{}" + "/target_{}/"
)

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

#model_colors = ['#DF9B1B', '#009682', '#4664AA', '#8CB6C3',   '#A22223', '#20b2aa']
"""
model_colors = ["#9b19f5", "#0bb4ff",#"#e6d800",
                "#ffa300", "#e60049", "#4A2377",
                #"#50e991", "#e6d800", 
                  #"#dc0ab4", 
                  "#b3d4ff", "#00bfa0"]

model_colors_2 = ["#f9e858", "#003a7d", "#008dff", "#ff73b6", "#b33dc6"]
"""
for i, area in enumerate(areas):
    scores = pd.DataFrame(
        index=targets,
        columns=[f"{config.model_combination}" for config in configs].append(
            "daily_profile"
        ),
        dtype="float",
    )

    for m, config in enumerate(configs):
        for j, targ in enumerate(targets):
            try:
                if m == 0:
                    y_test = pd.read_hdf(
                    version_folder_xgb.format(area, config.data_version)
                    + f"y_test.h5"
                ).loc[:, targ]
                else:
                    y_test = pd.read_hdf(
                        version_folder.format(area, config.data_version)
                        + f"y_test{config.scaled}.h5"
                    ).loc[:, targ]
                fit_folder = fit_folder_template.format(
                    area,
                    config.res_version,
                    config.model_combination,
                    config.scaler_str,
                    targ,
                )
                y_pred = pd.read_hdf(fit_folder + "y_pred.h5")
                if m == 0: # then this is xgboost
                    y_pred.rename(columns={"gtb_full": "predictions"}, inplace=True)
                    scores.loc[targ, "daily_profile"] = r2_score(
                        y_test, y_pred["daily_profile"]
                    )
                else:
                    y_pred.rename(
                        columns={f"{targ}_prediction": "predictions"}, inplace=True
                    )
                    
                scores.loc[targ, f"{config.model_combination}"] = r2_score(
                    y_test, y_pred["predictions"]
                )
            except Exception as e:
                print(e)
                print(
                    f"Error processing {config.model_combination} for {targ} in {area}: {e}"
                )
                continue

    # Replace NaN values with a placeholder (e.g., 0) for plotting
    scores = scores.fillna(0)

    # Use LaTeX for axis labels and titles
    scores.rename(columns={"daily_profile": "Daily Profile", "xgb": "XGBoost", "ngb": "NGBoost", "tabnet_tuner": "TabNet", "tabnet_proba_final":"TabNetProba"}, inplace=True)
    scores.plot.bar(
        ylim=[-0.02, scores.max().max() + 0.05],
        width=0.85,
        ax=ax[i],
        legend=False,
        #color=model_colors_2,
    )
    print(scores)
    ax[i].set_xticks(np.arange(len(targets)))
    ax[i].set_xticklabels(target_names, rotation=30, ha="right")
    ax[i].grid(axis="y")
    ax[i].set_title(area_names[i])

    relative_increase = scores.div(scores.iloc[:, 0], axis=0)

    """
    for j, targ in enumerate(targets):
        try:
            ml_gain = relative_increase.loc[targ].max()
            perf_val = scores.loc[targ, relative_increase.loc[targ].idxmax()]
            ax[i].annotate(
                r"$\times$ {:.1f}".format(ml_gain),
                (j - i * 0.1 - 0.2, perf_val + 0.015),
                fontsize=8,
                c=shap.plots.colors.gray_rgb,
                fontweight="bold",
            )
        except Exception as e:
            print(f"Error annotating {targ} in {area}: {e}")
            continue
    """

ax[0].set_ylabel(r"R$^2$ Score")  # LaTeX for R^2
ax[0].set_ylim([-0.01, 0.82])
ax[0].set_yticks(np.arange(0, 1.1, 0.2))  # Set y-axis ticks in 0.2 steps
ax[1].set_yticks(np.arange(0, 1.1, 0.2))  # Set y-axis ticks in 0.2 steps
plt.tight_layout()

"""
leg1 = ax[1].legend(bbox_to_anchor=(0.38, -0.2), ncol=3, title=r"Models")  # LaTeX for title
annot_text = plt.plot(
    -10, 0, marker=r"$\times$ 9.9", ms=22, lw=0, c=shap.plots.colors.gray_rgb
)
leg2 = ax[1].legend(
    annot_text, [r"Gain over daily \\ profile"], bbox_to_anchor=(0.9, -0.25)  # LaTeX for annotation
)
ax[1].add_artist(leg1)

"""

plt.legend(title=r"Models")
#plt.savefig("../../results/figures/" + 'r2_score_overview_proba.pdf', bbox_inches='tight')
#plt.savefig("../../results/figures/" + 'r2_score_xgb_tabnet.pdf', bbox_inches='tight')

plt.show()


# Figure 3. Actual vs predicted RoCoF Examples
## TODO: extract exactly the two days from the paper plot and not a fully random period

In [11]:
def get_prediction_df(config, area, target):
    X_train = pd.read_hdf(data_folder.format(area, config.data_version) + "X_train_full_scaled.h5")
    X_test = pd.read_hdf(data_folder.format(area, config.data_version) + "X_test_full_scaled.h5")

    y_train = pd.read_hdf(data_folder.format(area, config.data_version)+ f"y_train{config.scaled}.h5").loc[:, target]

    daily_profile = y_train.groupby(X_train.index.time).mean()
    daily_profile_std = y_train.groupby(X_train.index.time).std()

    y_pred = pd.read_hdf(fit_folder.format(area, config.res_version, config.model_combination, config.scaler_str, target) + "y_pred.h5")
    y_train_pred = pd.read_hdf(data_folder.format(area, config.data_version)+ f"y_train{config.scaled}.h5").loc[:, [target]]

    y_pred["daily_profile"] = [daily_profile[time] for time in X_test.index.time]
    y_pred["daily_profile_std"] = [daily_profile_std[time] for time in X_test.index.time]
    y_pred["crps_daily_profile"] =  ps.crps_gaussian(y_pred[f"{target}"],
                                                        np.array(y_pred["daily_profile"]),
                                                        np.array(y_pred["daily_profile_std"]))
    y_train_pred["daily_profile"] = [daily_profile[time] for time in X_train.index.time]
    y_train_pred["daily_profile_std"] = [daily_profile_std[time] for time in X_train.index.time]
    y_train_pred["crps_daily_profile"] =  ps.crps_gaussian(y_train_pred[f"{target}"],
                                                        np.array(y_train_pred["daily_profile"]),
                                                        np.array(y_train_pred["daily_profile_std"]))
    
    if config.model_combination == "ngb":
        ngb_model = joblib.load(fit_folder.format(area, config_ngb.res_version, config_ngb.model_combination, config_ngb.scaler_str, target) + "best_ngb_model.pkl")
        y_train_pred_dist = ngb_model.pred_dist(X_train)
        y_pred_dist = ngb_model.pred_dist(X_test)
        y_pred["crps_model"] = ps.crps_gaussian(y_pred[f"{target}"], y_pred_dist.params["loc"], y_pred_dist.params["scale"])

        y_train_pred["predictions_upper"] = y_train_pred_dist.dist.interval(0.95)[1]
        y_train_pred["predictions_lower"] = y_train_pred_dist.dist.interval(0.95)[0]
        y_train_pred["predictions"] = y_train_pred_dist.loc
        
        y_train_pred["crps_model"] = ps.crps_gaussian(y_train_pred[f"{target}"], y_train_pred_dist.params["loc"], y_train_pred_dist.params["scale"])
    
    elif config.model_combination == "tabnet_proba_final":
        confidence_level = 0.95
        z_score = 1.96  # z-score for 95% confidence interval
        
        y_pred.rename(columns={f"{target}_prediction": "predictions"}, inplace=True)
        
        reg = TabNetRegressorProba()
        reg.load_model(
            filepath=fit_folder.format(area, config.res_version, config.model_combination, config.scaler_str, target) + "best_model_params.zip")
        
        predictions = reg.predict(X_train.values)
        y_train_pred["predictions"] = predictions[:, 0]
        y_train_pred[f"{target}_cov_pred"] = predictions[:, 1]
        
        for df in [y_pred, y_train_pred]:
            std_dev = np.sqrt(df[f"{target}_cov_pred"])
            df["predictions_lower"] = df["predictions"] - z_score * std_dev
            df["predictions_upper"] = df["predictions"] + z_score * std_dev
            df["crps_model"] = ps.crps_gaussian(df[f"{target}"],
                                                np.array(df["predictions"]),
                                                np.array(std_dev)
                                            )


    y_train_pred_inverse = inverse_transform(y_train_pred, area, target, config)
    y_pred_inverse = inverse_transform(y_pred, area, target, config)

    y_pred_inverse['source'] = 'prediction'
    y_train_pred_inverse['source'] = 'training'

    y_inverse_full = pd.concat([y_pred_inverse, y_train_pred_inverse], axis=0)
    z_score = 1.96
    y_inverse_full["daily_profile_lower"] = y_inverse_full["daily_profile"] - z_score * y_inverse_full["daily_profile_std"] 
    y_inverse_full["daily_profile_upper"] = y_inverse_full["daily_profile"] + z_score * y_inverse_full["daily_profile_std"] 

    ##
    residual_std = np.std(y_inverse_full[f"{target}"] - y_inverse_full["daily_profile"])

    # Or if you already have this value from your model as "observation_noise" or similar

    # Calculate prediction interval
    prediction_std = np.sqrt(y_inverse_full["daily_profile_std"]**2 + residual_std**2)

    # Now calculate the prediction intervals
    y_inverse_full["daily_profile_lower"] = y_inverse_full["daily_profile"] - z_score * prediction_std
    y_inverse_full["daily_profile_upper"] = y_inverse_full["daily_profile"] + z_score * prediction_std
    ##
    
    y_inverse_full.drop(["mean_predictor", "std_predictor"], axis=1, inplace=True)
    return y_inverse_full.sort_index()

In [4]:
def plot_results(df, save=False, filename="plot.pdf", dpi=300, var_name="f_rocof", model_name=None, daily_profile_crps=None, model_crps=None):
    """
    Plot time series results with daily profile and predictions.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing the data to plot
    save : bool, default=False
        Whether to save the figure to disk
    filename : str, default="plot.pdf"
        Filename to use when saving the figure (can be .pdf, .png, etc.)
    dpi : int, default=300
        Resolution for the saved figure
    var_name : str, default="RoCoF"
        Name of the variable being plotted
    model_name : str, default=None
        Name of the model to display as a small title
    """
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    from matplotlib.patches import Patch
    import datetime
    
    # Enable LaTeX rendering
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
        "font.size": 14,
        "axes.labelsize": 16,
        "legend.fontsize": 14,
        "xtick.labelsize": 14,
        "ytick.labelsize": 14
    })
    
    # Generate automatic ylabel if none provided
    ylabel_var = target_names[var_name]
    unit = target_units[var_name]
    if unit == "":
        ylabel = r"$\textit{" + ylabel_var + r"~Value}$"
    else:
        ylabel = r"$\textit{" + ylabel_var + r"~Value}~[\mathrm{" + unit + r"}]$"
    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Make sure the dataframe is sorted by index (timestamp)
    df = df.sort_index()
    # Map the column names assuming they follow the pattern used in the original code
    actual_col = f"{var_name}"
    pred_col = "predictions"
    pred_lower_col = "predictions_lower"
    pred_upper_col = "predictions_upper"
    daily_profile_col = "daily_profile"
    daily_profile_lower_col = "daily_profile_lower"
    daily_profile_upper_col = "daily_profile_upper"

    daily_profile_crps = df.crps_daily_profile.mean()
    model_crps = df.crps_model.mean()

    
    # Plot the daily profile baseline with shading
    plt.plot(df.index, df[daily_profile_col],
             label='Daily Profile',
             color='purple',
             linestyle='-',
             alpha=0.7)
    
    # Add shading for daily profile
    plt.fill_between(df.index, 
                     df[daily_profile_lower_col], 
                     df[daily_profile_upper_col],
                     color='purple', alpha=0.1)
    
    # Plot actual values as a single continuous line for the entire dataset
    plt.plot(df.index, df[actual_col], 
             color='g', linestyle='-', linewidth=1.5, 
             alpha=0.8, label='_nolegend_')
    
    # Add a single line with the correct style for the legend
    var_name = var_name[2:]
    f_label = r'$f_{' + var_name + r'}$ actual'
    plt.plot([], [], color='g', linestyle='-', linewidth=1.5,label=f_label)
    
    # Plot a single line for all predictions regardless of source
    # Use legend_only plot for the legend entry
    f_pred_label = r'$f_{' + var_name + r'}$ predicted'
    plt.plot([], [], color='b', lw=1.5, label=f_pred_label)
    
    # Plot the continuous prediction line for the entire dataset
    plt.plot(df.index, df[pred_col], color='b', lw=1.5, label='_nolegend_')
    
    # Now handle the confidence intervals with different transparencies for training/prediction
    for start, end, source in get_chunks(df):
        chunk_data = df.loc[start:end]
        if source == 'training':
            # Confidence intervals for training
            plt.fill_between(chunk_data.index,
                            chunk_data[pred_lower_col],
                            chunk_data[pred_upper_col],
                            color='cornflowerblue', alpha=0.2)
        else:
            # Confidence intervals for test
            plt.fill_between(chunk_data.index,
                            chunk_data[pred_lower_col],
                            chunk_data[pred_upper_col],
                            color='cornflowerblue', alpha=0.5)
    
    # Add patches for the shadings
    daily_profile_patch = Patch(color='purple', alpha=0.2, label='Daily Profile PI')
    model_ci_patch_train = Patch(color='cornflowerblue', alpha=0.2, label=r'95\% PI (training)')
    model_ci_patch_test = Patch(color='cornflowerblue', alpha=0.5, label=r'95\% PI (test)')
    
    # Create a simpler legend with fewer elements but still separate CI entries
    handles, labels = ax.get_legend_handles_labels()
    
    # Add the confidence interval patches to the legend
    handles.append(daily_profile_patch)
    handles.append(model_ci_patch_train)
    handles.append(model_ci_patch_test)
    labels.append(r'95\% PI (daily profile)')
    labels.append(r'95\% PI (training)')
    labels.append(r'95\% PI (test)')
    
    # Create the new legend and position it outside the plot
    # Place the legend outside the plot area at the bottom
    ax.legend(handles, labels, fontsize=12, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
    
    # Extract unique dates from the index
    from datetime import datetime
    import pandas as pd
    import numpy as np
    
    # Get the date part only from the datetime index and find unique dates
    date_strings = [d.strftime('%Y-%m-%d') for d in df.index]
    unique_date_strs = list(set(date_strings))
    unique_dates = [datetime.strptime(d, '%Y-%m-%d') for d in unique_date_strs]
    unique_dates.sort()  # Sort chronologically
    print(unique_dates)
    # Set these specific dates as tick locations
    ax.set_xticks(unique_dates)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    #ax.tick_params(axis='x', labelrotation=40)
    
    # LaTeX formatted labels
    plt.ylabel(ylabel, size=16)
    plt.grid(True, alpha=0.3)
    
    # Create a box for model information and metrics in the top right
    info_text = ""
    if model_name:
        info_text += r"\textit{Model: " + model_name + "}"
    
    # Add metrics information if provided
    if model_crps is not None or daily_profile_crps is not None:
        if info_text:
            info_text += r"\\"  # Line break in LaTeX
        
        if model_crps is not None:
            info_text += r"Model CRPS: " + f"{model_crps:.2f}"
        
        if daily_profile_crps is not None:
            if model_crps is not None:
                info_text += r"\\"  # Line break in LaTeX
            info_text += r"Daily Profile CRPS: " + f"{daily_profile_crps:.2f}"
    
    # If we have any info to display, create the text box
    if info_text:
        ax.text(0.99, 0.98, info_text, 
                transform=ax.transAxes, 
                fontsize=10, 
                ha='right', 
                va='top',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7),
                multialignment='right')
    
    # Adjust bottom margin to make room for the legend
    plt.subplots_adjust(bottom=0.25)
    plt.tight_layout(rect=[0, 0.1, 1, 0.95])
    
    # Save the figure if requested
    if save:
        plt.savefig(filename, dpi=dpi, bbox_inches='tight')
        print(f"Figure saved as '{filename}' with DPI={dpi}")
    
    plt.show()

# Helper function to find chunks of training and prediction data
def get_chunks(df):
    """
    Find continuous chunks of the same source type (training or prediction)
    in a sorted dataframe. Returns a list of tuples (start_index, end_index, source_type).
    """
    # Make sure the dataframe is sorted
    df = df.sort_index()
    
    chunks = []
    current_source = None
    start_idx = None
    for idx, row in df.iterrows():
        if current_source is None:
            current_source = row['source']
            start_idx = idx
        elif row['source'] != current_source:
            chunks.append((start_idx, idx, current_source))
            current_source = row['source']
            start_idx = idx
    
    # Add the last chunk
    if current_source is not None and start_idx is not None:
        chunks.append((start_idx, df.index[-1], current_source))
    
    return chunks

In [15]:
def extract_random_period(df, period_length):
    """
    Extract a random period of specified length from a datetime-indexed DataFrame.
    The function ensures the extracted period always starts at the beginning of a day (00:00:00)
    and ends at the end of a day (23:59:59.999999), and handles timezone-aware datetime objects.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame with datetime index
    period_length : str or timedelta
        Length of the period to extract (e.g., '7D' for 7 days, '3M' for 3 months)
        Can be a string compatible with pandas.Timedelta or a timedelta object
        
    Returns:
    --------
    tuple
        A tuple containing (random_start, random_end) datetime objects
        random_start will be at midnight (00:00:00) and random_end will be at end of day (23:59:59.999999)
        Both preserve timezone information
    """
    if isinstance(period_length, str):
        period_length = pd.Timedelta(period_length)
    
    # Get the min and max dates of the dataframe
    start_date = df.index.min()
    end_date = df.index.max()
    is_tz_aware = start_date.tzinfo is not None
    
    # Convert to midnight of first day
    if is_tz_aware:
        tz = start_date.tzinfo
        start_date_midnight = pd.Timestamp.combine(start_date.date(), time(0, 0, 0)).tz_localize(tz)
        # End date is the midnight of the next day after the last data point
        end_date_midnight = pd.Timestamp.combine(end_date.date(), time(0, 0, 0)).tz_localize(tz)
    else:
        start_date_midnight = datetime.combine(start_date.date(), time(0, 0, 0))
        end_date_midnight = datetime.combine(end_date.date(), time(0, 0, 0))
    
    # Calculate total range (in days)
    # We need to adjust to get full days
    days_in_period = period_length.days
    
    if days_in_period <= 0:
        raise ValueError(f"Period length must be at least 1 day, got {period_length}")
    
    # Calculate the latest possible start date to have a full period
    latest_start_date = end_date_midnight - pd.Timedelta(days=days_in_period-1)
    
    # Calculate total days available for random selection
    days_available = (latest_start_date - start_date_midnight).days
    
    if days_available < 0:
        raise ValueError(f"Requested period length ({period_length}) is longer than available data range")
    
    # Generate random start day
    random_days = random.randint(0, days_available)
    random_start = start_date_midnight + pd.Timedelta(days=random_days)
    
    # Calculate end date (end of the last day in the period)
    random_end_date = (random_start + pd.Timedelta(days=days_in_period-1)).date()
    
    # Set random_end to the end of the day (23:59:59.999999)
    if is_tz_aware:
        end_of_day = pd.Timestamp.combine(
            random_end_date, 
            time(23, 59, 59, 999999)
        ).tz_localize(tz)
    else:
        end_of_day = datetime.combine(
            random_end_date, 
            time(23, 59, 59, 999999)
        )
    
    return random_start, end_of_day

In [ ]:
df = get_prediction_df(config_ngb, "CE", "f_rocof")
rand_start, rand_end = extract_random_period(df, '2D')
rand_end

In [ ]:
for config in [config_ngb, config_tabnet_proba]:
    df = get_prediction_df(config, area, "f_rocof")
    random_period = df.loc[random_start:random_end]
    filename = f"figures_crps/{area}_{target}_{config.model_name}.pdf"
    plot_results(df=random_period, save=True, filename=filename, var_name="f_roxof", model_name=config.model_name)

# Figure 4, 5, 6, 8. Relative SHAP Feature Importance Plot 
* Mean Prediction CE and Nordic

In [19]:
from scipy.cluster.hierarchy import fcluster, linkage
import scipy.cluster.hierarchy as sch

def get_clustering(area, config, cutoff):
    X_train = pd.read_hdf(
                        data_folder.format(area, config.data_version, config.scaler_str)
                        + f"X_train_full{config.scaled}.h5")
    X_train.rename(columns=input_col_names, inplace=True)

    correlation_matrix = X_train.corr(method='pearson').abs()
    dist_matrix = 1 - correlation_matrix

    clustering = linkage(dist_matrix, method="complete", optimal_ordering=True)
    clusters_cutoff = fcluster(clustering, cutoff, criterion='distance')
    name_to_cluster = {name: cluster for name, cluster in zip(X_train.columns, clusters_cutoff)}
   
    return clustering, name_to_cluster


def get_shap_df(area, config, target, shap_type):
    X_test = pd.read_hdf(
                        data_folder.format(area, config.data_version, config.scaler_str)
                        + f"X_test_full{config.scaled}.h5")
    X_test.rename(columns=input_col_names, inplace=True)
    shap_vals = np.load(
            explain_folder.format(area, config.res_version, config.model_combination, target, config.explanations)
            + f"shap_values_{shap_type}.npy"
        )
    if len(shap_vals.shape) == 3 and shap_vals.shape[2] == 1:
        shap_vals = shap_vals.reshape(shap_vals.shape[0], shap_vals.shape[1])

    shap_vals = pd.DataFrame(
                    data=shap_vals, index=X_test.index, columns=X_test.columns
                )
    return shap_vals

In [20]:
import scipy.cluster.hierarchy as sch
def plot_shap_comparison_partition(configs: list[Config], area: str, shap_type: str, target: str, cutoff: float, save: bool = False):

    shap_mean_importances: dict[str, dict[str, float]] = defaultdict(dict)
    clustering, name_to_cluster = get_clustering(area,configs[0], cutoff=cutoff)
    
    names_by_cluster = defaultdict(list)

    
    for name, cluster in name_to_cluster.items():
        names_by_cluster[cluster].append(name)

    figpath_name = f"../results/figures/comparison_plots/shap_comparison_{area}_{target}_{shap_type}_{str(cutoff)}"
    for conf in configs:
        figpath_name += "_" + conf.model_name
        shap_values = get_shap_df(area, conf, target, shap_type)

        total_shap = 0 
        for feature, shap_value in shap_values.items():
            total_shap += shap_value.abs().mean()
        for feature, shap_value in shap_values.items():
            cluster = name_to_cluster[feature]
            shap_mean_importances[cluster][conf.model_name] = shap_mean_importances[cluster].get(conf.model_name, 0) + shap_value.abs().mean() / total_shap

    clusters = list(shap_mean_importances.keys())
    clusters.sort(key= lambda x: max(shap_mean_importances[x].values()), reverse=True)

    #for cluster in clusters:
    #    print("\nCluster ", cluster)
    #    for feature in names_by_cluster[cluster]:
    #       print(f"\t{feature}")

    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')

    markers_by_model = {"XGBoost": "x", "NGBoost": "o", "TabNet": "+", "TabnetProba": "*"}
    colors_by_model = {"XGBoost": "tab:blue", "NGBoost": "tab:orange", "TabnetProba": "tab:green", "TabNet": "tab:red"}
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.grid(True, which='both', axis='both', color='lightgrey', linestyle='-', linewidth=0.5)

    #fig, ax = plt.figure()
    k = 10

    for i, cluster in enumerate(reversed(clusters[:k])):
        for model, value in shap_mean_importances[cluster].items():
            model_name = str(model)
            if not i:
                ax.scatter(value, i, c=colors_by_model[model], marker=markers_by_model[model], label=model_name)
            else:
                ax.scatter(value, i, c=colors_by_model[model], marker=markers_by_model[model])

    ax.legend()
    others = clusters[k:] 
    
    clusters_labels = defaultdict(str)
    # Create cluster names
    if area == "CE":
        cluster_mappings= {
            frozenset(["Gas ramp", "Reservoir hydro ramp", "Run-off-river hydro ramp", "Hard coal ramp"]): "Gas, hydro and hard coal ramps",
            frozenset(["Biomass ramp", "Pumped hydro ramp", "Price ramp day-ahead"]): "Biomass, pumped hydro and prices DA ramps",
            frozenset(["Load ramp", "Total generation ramp", "Load ramp day-ahead", "Generation ramp day-ahead"]): "Load and generation ramps cluster",
            frozenset(["Solar ramp", "Forecast error generation ramp", "Solar ramp day-ahead"]): "Solar and forecast error gen. ramps cluster",
            frozenset(["Solar generation", "Solar day-ahead"]): "Solar generation cluster",
            frozenset(["Solar ramp", "Solar ramp day-ahead"]): "Solar ramp cluster",
            frozenset(["Biomass generation", "Oil generation"]): "Biomass and oil gen. cluster",
            #frozenset(["Nuclear generation", "Other renewable generation"]): "Other renewable gen. cluster",
            frozenset(["Biomass generation", "Other renewable generation"]): "Other renewable gen. and biomass cluster",
        }
    elif area == "Nordic":
        cluster_mappings= {
            frozenset(["Load ramp", "Total generation ramp", "Run-off-river hydro ramp","Reservoir hydro ramp","Load ramp day-ahead", "Generation ramp day-ahead"]): "Load, gen. and hydro ramps cluster",
            frozenset(["Load", "Reservoir hydro generation","Total generation","Synchronous generation","Scheduled generation","Load day-ahead"]): "Load, reservoir hydro and generation cluster",
            frozenset(["Onshore wind ramp", "Onshore wind ramp day-ahead"]): "Onshore wind ramps cluster",
            frozenset(["Solar generation", "Forecast error solar","Solar day-ahead"]): "Solar features cluster",
            frozenset(["Offshore wind ramp", "Forecast error offshore wind ramp"]): "Offshore wind cluster"
        }
    
    for cl in clusters:
            cluster_features = frozenset(names_by_cluster[cl])
            if cluster_features in cluster_mappings.keys():
                print("Cluster features: ")
                print(cluster_features)
                print("Cluster mappings: ")
                print(cluster_mappings[cluster_features])
                clusters_labels[cl] = cluster_mappings[cluster_features]
            
            else:
                clusters_labels[cl] = names_by_cluster[cl][0]

    clusters_names = list(clusters_labels.values())
    ax.set_yticks(list(range(k+1)), list(reversed(clusters_names[:k])) + [f"Sum of {len(others)} other features"], fontsize=12)

    others_by_model = defaultdict(float)
    for cluster in others:
        for model, value in shap_mean_importances[cluster].items():
            others_by_model[model] += value

    for model in shap_mean_importances[cluster].keys():
        ax.scatter(others_by_model[model], i+1, c=colors_by_model[model], marker=markers_by_model[model])

    plt.title(r'\textbf{SHAP Feature Importance for} \textit{' + target + r'} \textbf{in ' + area + '}', fontsize=14)
    if shap_type == "mean":
        plt.xlabel(r'Relative SHAP value (impact on mean prediction)', fontsize=14)
    else:
        plt.xlabel(r'Relative SHAP value (impact on uncertainty prediction)', fontsize=14)

    figpath_name +=".pdf"
    if save:
        save_path = figpath_name
        plt.savefig(save_path, bbox_inches='tight')  # Save the figure
        print(f"Figure saved at {save_path}")
        plt.show()
    else:
        plt.show()
    plt.show()

configs_det = [config_xgb, config_tabnet]
shap_type = "mean"
area="CE"
target="f_rocof"
if area == "CE":
    cutoff=0.8
else:
    cutoff=0.85

configs_det = [config_xgb, config_tabnet]
configs_proba = [config_ngb, config_tabnet_proba]
configs_all = [config_xgb, config_tabnet, config_tabnet_proba, config_ngb]

In [ ]:
plot_shap_comparison_partition(configs_proba, area, shap_type, target, cutoff=cutoff, save=False)

* Uncertainty Prediction CE and Nordic

# Figure 7, 9. Beeswarm Plot Uncertainty Prediction

In [21]:
import matplotlib.pyplot as plt
import shap
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

from utils.plot_beeswarm import my_beeswarm

def plot_shap_beeswarm_grid(configs, area, target, shap_type, names_by_cluster, 
                            sorted_clusters, num_clusters, save=False, clip_outliers=False):
    X_test = pd.read_hdf(
        data_folder.format(area, configs[0].data_version, configs[0].scaler_str)
        + f"X_test_full{configs[0].scaled}.h5"
    )
    X_test.rename(columns=input_col_names, inplace=True)

    num_models = len(configs)
    size_const = 3.5
    if num_clusters > 3:
        size_const = 3
    fig, axs = plt.subplots(nrows=num_clusters, ncols=num_models, figsize=(size_const*num_models, 2*num_clusters), 
                            squeeze=False, constrained_layout=True)
    config_str = "".join([c.model_name for c in configs])
    
    # Define a custom colormap from blue to purple to red, similar to SHAP's
    colors = ["#1f77b4", "#9467bd", "#ff007f"]  # Blue -> Purple -> Red
    cmap = LinearSegmentedColormap.from_list("shap_custom", colors)
    
    for row, cluster in enumerate(sorted_clusters[:num_clusters]):
        for col, config in enumerate(configs):
            features_in_cluster = names_by_cluster[cluster]
            
            # Get SHAP values for the current config and cluster
            shap_values = get_shap_df(area, config, target, shap_type)
            top_shap_vals = shap_values[features_in_cluster]
            cluster_shap_values = shap.Explanation(top_shap_vals.values, feature_names=features_in_cluster, data=X_test[features_in_cluster])
            
            ax = axs[row, col]
            my_beeswarm(ax=ax, shap_values=cluster_shap_values, show=False, plot_size=None, labels=not col, clip_outliers=clip_outliers)
            ax.set_title(f"Cluster {row + 1} - {config.model_name}", fontsize=12)

    # Create the color bar using the custom cmap
    m = plt.cm.ScalarMappable(cmap=cmap)
    m.set_array([0, 1])
    
    # Add the color bar on the side for a shared legend
    cb = fig.colorbar(m, ax=axs, ticks=[0, 1], aspect=80, orientation="vertical", fraction=0.2, pad=0.04)
    cb.set_ticklabels(["Low", "High"])
    cb.set_label("Feature Value", size=12, labelpad=0)
    cb.ax.tick_params(labelsize=11, length=0)
    cb.set_alpha(1)
    cb.outline.set_visible(False)

    figpath_name = f"../../results/figures/comparison_plots/beeswarm/{area}_{shap_type}_{config_str}_beeswarms{num_clusters}.pdf"
    if save:
        plt.savefig(figpath_name, bbox_inches='tight')
        print(f"Figure saved at {figpath_name}")
    
    plt.show()


In [22]:
def get_sorted_clusters(configs: list[Config], area: str, shap_type: str, target: str, cutoff: float):
    shap_mean_importances: dict[str, dict[str, float]] = defaultdict(dict)
    clustering, name_to_cluster = get_clustering(area, configs[0], cutoff)
    
    names_by_cluster = defaultdict(list)
    
    for name, cluster in name_to_cluster.items():
        names_by_cluster[cluster].append(name)

    for conf in configs:
        shap_values = get_shap_df(area, conf, target, shap_type)
  
        total_shap = 0 
        for feature, shap_value in shap_values.items():
            total_shap += shap_value.abs().mean()

        for feature, shap_value in shap_values.items():
            cluster = name_to_cluster[feature]
            shap_mean_importances[cluster][conf.model_name] = shap_mean_importances[cluster].get(conf.model_name, 0) + shap_value.abs().mean() / total_shap
    
    sorted_clusters = list(shap_mean_importances.keys())
    sorted_clusters.sort(key= lambda x: max(shap_mean_importances[x].values()), reverse=True)
    
    return clustering, sorted_clusters, names_by_cluster


In [ ]:
# Beeswarm Plot for CE RoCoF
shap_type = "std"
area="CE"
target="f_rocof"
cutoff = 0.8

num_clusters = 1# Set number of clusters to display
clustering, sorted_clusters, names_by_cluster = get_sorted_clusters(configs_proba, area, shap_type, target, cutoff=cutoff)
plot_shap_beeswarm_grid(configs_proba, area, target, shap_type, names_by_cluster, sorted_clusters, num_clusters, save=False, clip_outliers=False)


In [ ]:
# Beeswarm Plot for Nordic Nadir
shap_type = "std"
area="Nordic"
target="f_ext"
cutoff = 0.85

num_clusters = 2# Set number of clusters to display
clustering, sorted_clusters, names_by_cluster = get_sorted_clusters(configs_proba, area, shap_type, target, cutoff=cutoff)
plot_shap_beeswarm_grid(configs_proba, area, target, shap_type, names_by_cluster, sorted_clusters, num_clusters, save=False, clip_outliers=False)


# Appendix
## Figure 10 - Correlation Analysis

In [ ]:
def plot_double_correlation_heatmap(save=False, threshold=0.7):
    """
    Plots a double heatmap for the CE and Nordic areas, showing only correlations
    with absolute value greater than or equal to the specified threshold.
    
    Parameters:
    - threshold: float, the threshold for filtering correlations (default: 0.7).
    """
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    data_version = "2024-05-19"
    
    areas = ["CE", "Nordic"]
    figure_path = "../probabilistic-XAI-for-grid-frequency-stability/results/figures/correlation/"
    
    if not os.path.exists(figure_path):
        os.makedirs(figure_path)
        
    _, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 16), sharex=True)

    for i, area in enumerate(areas):
        version_folder = f"../data/2020-2024/{area}/version_{data_version}/yeo_johnson/"
        X_train = pd.read_hdf(version_folder + "X_train_full_scaled.h5")
        X_train.rename(columns=input_col_names, inplace=True)

        # Calculate and filter correlation matrix
        correlation_matrix = X_train.corr(method='pearson')
        filtered_corr_matrix = correlation_matrix.copy()
        filtered_corr_matrix[np.abs(filtered_corr_matrix) < threshold] = 0

        non_zero_columns = filtered_corr_matrix.columns[
            (np.abs(filtered_corr_matrix).sum(axis=0) - np.abs(np.diag(filtered_corr_matrix))) > 0
        ]
        filtered_corr_matrix = filtered_corr_matrix.loc[non_zero_columns, non_zero_columns]

        sns.heatmap(
            filtered_corr_matrix, annot=False, cmap="coolwarm", ax=axes[i],
            linewidths=.5, vmin=-1, vmax=1, cbar_kws={'label': 'Pearson Correlation' if i == 1 else None}
        )
        
        axes[i].set_title(r'Correlation Heatmap of ' + area + r' Features ($|corr| \geq ' + str(threshold) + r'$)', fontsize=16)
        axes[i].tick_params(axis='x', rotation=45)
        for tick in axes[i].get_xticklabels():
            tick.set_horizontalalignment('right')
            tick.set_verticalalignment('center_baseline')
            
    plt.tight_layout()
    if save:
        plt.savefig(figure_path + "double_corr_heatmap.pdf", bbox_inches='tight')
    plt.show()

plot_double_correlation_heatmap(threshold=0.7, save=False)


## Figure 12 - Performance Overview of All Models

## Figure 13, 14 - Calibration Plots

In [ ]:

def plot_calibration_curves(area, save: bool):
    """
    Function to plot calibration curves for different areas and targets based on the provided config.
    
    Args:
    - config: An object containing necessary configurations such as data folder paths, 
              list of areas, targets, and other model-related settings.
    """
    plt.rcParams['text.usetex'] = True
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.size'] = 12

    areas = ["CE", "Nordic"]
    targets = ["f_rocof", "f_ext", "f_msd", "f_integral"]

    target_mappings = {
        "f_rocof": "RoCoF",
        "f_ext": "Nadir",
        "f_msd": "MSD",
        "f_integral": "Integral"
    }

    confidence_levels = np.arange(0.1, 1.0, 0.1).tolist() + [0.99]  
    z_values = [norm.ppf((1 + level) / 2) for level in confidence_levels]  
    fig, axes = plt.subplots(1, 2, figsize=(11, 5))
    axes = dict(zip(["NGBoost", "TabnetProba"], axes))  

    for config in [config_ngb, config_tabnet_proba]:
        for target in targets:
            X_test = pd.read_hdf(data_folder.format(area, config.data_version) + "X_test_full_scaled.h5")
            y_test = pd.read_hdf(data_folder.format(area, config.data_version) + f"y_test{config.scaled}.h5").loc[:, target]
            y_pred = pd.read_hdf(fit_folder.format(area, config.res_version, config.model_combination, config.scaler_str, target) + "y_pred.h5")
            if config.model_name == "NGBoost":

                ngb_model = joblib.load(fit_folder.format(area, config.res_version, config.model_combination, config.scaler_str, target) + "best_ngb_model.pkl")

                y_pred_ngb = ngb_model.pred_dist(X_test)
                y_pred_mean = y_pred["predictions"]
                y_pred_std = y_pred_ngb.scale
            elif config.model_name == "TabnetProba":
                model = TabNetRegressorProba()
                model.load_model(fit_folder.format(area, config.res_version, config.model_combination, config.scaler_str, target) + "best_model_params.zip")
                y_pred_tabnet = model.predict(X_test.values)
                y_pred_mean = y_pred_tabnet[:, 0]
                y_pred_std = np.sqrt(y_pred_tabnet[:, 1]) + 1e-6
            
            y_true = y_test.values

            observed_coverage = []

            for z in z_values:
                lower_bounds = y_pred_mean - z * y_pred_std
                upper_bounds = y_pred_mean + z * y_pred_std
                within_interval = np.logical_and(y_true >= lower_bounds, y_true <= upper_bounds)
                observed_coverage.append(np.mean(within_interval))

            axes[config.model_name].plot(confidence_levels, observed_coverage, marker='o', label=target_mappings[target])

        axes[config.model_name].plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect calibration')
        axes[config.model_name].set_xlabel('Expected confidence level', fontsize=14)
        axes[config.model_name].set_ylabel('Observed coverage', fontsize=14)
        axes[config.model_name].set_title(f'Prediction Interval Calibration - {config.model_name}')
        axes[config.model_name].legend(title="Targets", fontsize=12)

    plt.tight_layout()
    if save:
        figpath_name = f"../../results/figures/comparison_plots/calibration_curve_{area}.pdf"
        plt.savefig(figpath_name, bbox_inches='tight')  # Save the figure
        print(f"Figure saved at {figpath_name}")
        plt.show()
    else:
        plt.show()

#plot_calibration_curves("CE", save=False)

#plot_calibration_curves("Nordic", save=False)


## Figure 15, 16 - Clustering Dendrograms

In [25]:
def plot_corr_clustering_cutoff(area,  cutoff : float,  data_version="2024-05-19", input_cols=None, save=False):
    version_folder = f"../data/2020-2024/{area}/version_{data_version}/yeo_johnson/"
    X_train = pd.read_hdf(version_folder + "X_train_full_scaled.h5")
    
    figure_path = (
        f"../../probabilistic-XAI-for-grid-frequency-stability/results/figures/correlation/{area}/"
    )
    
    if not os.path.exists(figure_path):
        os.makedirs(figure_path)
    
    correlation_matrix = X_train.corr(method='pearson')

    correlation_matrix_abs = np.abs(correlation_matrix)
    dist_matrix_df = 1 - correlation_matrix_abs

    clustering = sch.linkage(dist_matrix_df, method="complete", optimal_ordering=True)
    
    if input_cols:
        labels = [input_cols.get(col, col) for col in X_train.columns]
    else:
        labels = X_train.columns
    
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    
    plt.figure(figsize=(15, 7))
    plt.title(r"\textbf{Dendrogram of Correlation Clustering of " + area + ' Features}', fontsize=14)
    plt.axhline(y=cutoff, c='k', lw=1.5, linestyle='--')
    plt.text(0.95, cutoff, f'cutoff:{cutoff}', va='center_baseline', ha='right', color='k', fontsize=10)

    
    plt.xticks(rotation=90)
    plt.tick_params(axis='x', which='major', labelsize=12)
    plt.ylabel(r'\textbf{Correlation Distance}', fontsize=14)
    
    if save:
        plt.savefig(figure_path + f"pearson_corr_{area}_{cutoff}.pdf", bbox_inches='tight')
    plt.show()
    
    return clustering


In [ ]:
clustering_Nordic= plot_corr_clustering_cutoff(area="Nordic", cutoff=0.85, input_cols=input_col_names, save=False)
clustering_CE = plot_corr_clustering_cutoff(area="CE", cutoff=0.8, input_cols=input_col_names, save=False)
